In [205]:
import math
from pprint import pprint
from math import log, factorial, sqrt
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
    permutations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter, xor, and_, or_, indexOf
from bisect import bisect_right, bisect_left
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import sets
import matrix
import mathematics
import sequences
from importlib import reload

for m in (
    arrays,
    trees,
    lists,
    graphs,
    search,
    strings,
    sets,
    sequences,
    matrix,
    mathematics,
):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import sets
import matrix as mt
import mathematics as m
from sequences import find_if

print()

In [208]:
def rindex(a: list, v) -> int:
    "Return the rear index of `v` in `a`."
    return len(a) - indexOf(reversed(a), v) - 1


def two_repeated_numbers(a: List[int]) -> Tuple[int, int]:
    "Return two numbers that repeat in `a`."
    n = len(a) - 2
    s = -n * (n + 1) // 2
    p = -n * (n + 1) * (2 * n + 1) // 6
    for e in a:
        s += e
        p += e * e

    b = -s
    c = (s * s - p) // 2

    r = int(sqrt(b * b - 4 * c))
    x = max(-b + r, -b - r) // 2
    y = s - x

    xi, yi = rindex(a, x), rindex(a, y)

    if yi < xi:
        x, y = y, x

    return x, y

In [246]:
list(islice([1, 2, 3], 3 - 1))

[1, 2]

In [251]:
def assign(s, v, i):
    f = s[i] = v
    return f


def two_repeated_numbers(a: List[int]) -> Tuple[int, int]:
    "Return repeated numbers in `a`."

    def repeated(e):
        f = a[abs(e)] = -a[abs(e)]
        return f > 0 and abs(e)

    return tuple(filter(None, map(repeated, a)))

In [252]:
two_repeated_numbers([1, 2, 3, 2, 3, 4, 5])

(2, 3)

In [211]:
4 * 3 // 2

6

In [253]:
tuple([])

()